In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Импорт библиотек

In [ ]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, \
    recall_score, f1_score, log_loss, auc, classification_report, confusion_matrix, \
    precision_recall_curve, roc_curve

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings

warnings.filterwarnings("ignore")
RAND = 10
percent_of_negative_class = 0.97

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Метод для подсчёта метрик

In [ ]:
def get_metrics(y_test, y_pred, y_score, name):
    df_metrics = pd.DataFrame()
    
    df_metrics['model'] = [name]
    
    df_metrics['Accuracy'] = [accuracy_score(y_test, y_pred)]
    df_metrics['ROC_AUC'] = [roc_auc_score(y_test, y_score[:,1])]
    df_metrics['Precision'] = [precision_score(y_test, y_pred)]
    df_metrics['Recall'] = [recall_score(y_test, y_pred)]
    df_metrics['f1'] = [f1_score(y_test, y_pred)]
    df_metrics['Logloss'] = [log_loss(y_test, y_score)]
    
    return df_metrics

# Подготовка данных к обучению

Выгрузим данные, сохранённые на этапе EDA

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/data_test3.parquet.gzip')
df.head()

,SEMESTER,DISC_ID,TYPE_NAME,DEBT,GENDER,CITIZENSHIP,EXAM_TYPE,EXAM_SUBJECT_1,EXAM_SUBJECT_2,EXAM_SUBJECT_3,...,DISC_DEP_12779834774062657273,DISC_DEP_12795149246808839444,DISC_DEP_12866670834530293829,DISC_DEP_12896073176567118977,DISC_DEP_13705271043836613455,DISC_DEP_16131140458546037814,DISC_DEP_16828277449727897492,DISC_DEP_17522523368314118110,DISC_DEP_18446744073709551615,mean_score
0,1,10502311854018326223,Зачет,0,М,15601729049989747827,ЕГЭ,70786669040476600,5533732657842394915,8388269026169219461,...,0,0,0,0,0,0,0,0,0,82.666667
1,1,1601392918367593206,Зачет,0,М,15601729049989747827,ЕГЭ,70786669040476600,5533732657842394915,8388269026169219461,...,0,0,0,0,0,0,0,0,0,82.666667
2,1,9559803959325174929,Зачет,0,М,15601729049989747827,ЕГЭ,70786669040476600,5533732657842394915,8388269026169219461,...,0,0,0,0,0,0,0,0,0,82.666667
3,1,8955667882044263414,Зачет,0,М,15601729049989747827,ЕГЭ,70786669040476600,5533732657842394915,8388269026169219461,...,0,0,0,0,0,0,0,0,0,82.666667
4,1,17741967398854095262,Экзамен,0,М,15601729049989747827,ЕГЭ,70786669040476600,5533732657842394915,8388269026169219461,...,0,0,0,0,0,0,0,0,0,82.666667


## Бинаризация

Проведём бинаризацию датафрейма

In [ ]:
df_bin = pd.get_dummies(df, drop_first=True)
df_bin.head()

,SEMESTER,DISC_ID,DEBT,CITIZENSHIP,EXAM_SUBJECT_1,EXAM_SUBJECT_2,EXAM_SUBJECT_3,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,...,TYPE_NAME_Курсовой проект,TYPE_NAME_Экзамен,GENDER_М,EXAM_TYPE_ЕГЭ,EXAM_TYPE_ОЛИМПИАДА,ADMITTED_SUBJECT_PRIZE_LEVEL_1.0,ADMITTED_SUBJECT_PRIZE_LEVEL_2.0,ADMITTED_SUBJECT_PRIZE_LEVEL_3.0,ADMITTED_SUBJECT_PRIZE_LEVEL_4.0,ADMITTED_SUBJECT_PRIZE_LEVEL_ЕГЭ
0,1,10502311854018326223,0,15601729049989747827,70786669040476600,5533732657842394915,8388269026169219461,78.0,79.0,91.0,...,0,0,1,1,0,0,0,0,0,1
1,1,1601392918367593206,0,15601729049989747827,70786669040476600,5533732657842394915,8388269026169219461,78.0,79.0,91.0,...,0,0,1,1,0,0,0,0,0,1
2,1,9559803959325174929,0,15601729049989747827,70786669040476600,5533732657842394915,8388269026169219461,78.0,79.0,91.0,...,0,0,1,1,0,0,0,0,0,1
3,1,8955667882044263414,0,15601729049989747827,70786669040476600,5533732657842394915,8388269026169219461,78.0,79.0,91.0,...,0,0,1,1,0,0,0,0,0,1
4,1,17741967398854095262,0,15601729049989747827,70786669040476600,5533732657842394915,8388269026169219461,78.0,79.0,91.0,...,0,1,1,1,0,0,0,0,0,1


## Разбиение на train/test

Разобьём датафрейм на обучающую и тестовую выборки для обоих датафреймов (бинаризованного и небинаризованного, чтобы в зависимости от модели подавать на вход тот или иной вариант)

In [ ]:
feature_cols = df.drop(columns = ['DEBT'])

X_train, X_test, y_train, y_test = train_test_split(feature_cols,
                                                    df['DEBT'],
                                                    test_size=0.16,
                                                    random_state=10)

In [ ]:
feature_cols_bin = df_bin.drop(columns = ['DEBT'])

X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(feature_cols_bin,
                                                    df_bin['DEBT'],
                                                    test_size=0.16,
                                                    random_state=10)

## Нормализация

Приведём все признаки к одной шкале с помощью MinMaxScaler для бинаризованных данных - это будет третья и последняя вариация формата наших выборок

In [ ]:
st = MinMaxScaler()
X_train_bin_scaled = st.fit_transform(X_train_bin)
X_test_bin_scaled = st.transform(X_test_bin)

# Logistic regression

Приступим к обучению наших бейзлайн моделей

In [ ]:
lr = LogisticRegression(class_weight='balanced', random_state=RAND)
lr.fit(X_train_bin_scaled, y_train)

LogisticRegression(class_weight='balanced', random_state=10)

In [ ]:
y_pred = lr.predict(X_test_bin_scaled)
y_pred_prob = lr.predict_proba(X_test_bin_scaled)
metrics = get_metrics(y_test, y_pred, y_pred_prob, name='LogisticRegression')

y_pred_train = lr.predict(X_train_bin_scaled)
y_pred_prob_train = lr.predict_proba(X_train_bin_scaled)
metrics = metrics.append(
    get_metrics(y_train, y_pred_train, y_pred_prob_train, name='LogisticRegression_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340


# Decision tree

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced', random_state=RAND)
dt.fit(X_train_bin, y_train_bin)

DecisionTreeClassifier(class_weight='balanced', random_state=10)

In [ ]:
y_pred = dt.predict(X_test_bin)
y_pred_prob = dt.predict_proba(X_test_bin)
metrics = metrics.append(
    get_metrics(y_test_bin, y_pred, y_pred_prob, name='Decision_tree'))

y_pred_train = dt.predict(X_train_bin)
y_pred_prob_train = dt.predict_proba(X_train_bin)
metrics = metrics.append(
    get_metrics(y_train, y_pred_train, y_pred_prob_train, name='Decision_tree_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193


Видим переобучение

# Random forest

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', bootstrap=False, random_state=RAND)
rf.fit(X_train_bin, y_train_bin)

RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       random_state=10)

In [ ]:
y_pred = rf.predict(X_test_bin)
y_pred_prob = rf.predict_proba(X_test_bin)
metrics = metrics.append(
    get_metrics(y_test_bin, y_pred, y_pred_prob, name='Random_forest'))

y_pred_train = rf.predict(X_train_bin)
y_pred_prob_train = rf.predict_proba(X_train_bin)
metrics = metrics.append(
    get_metrics(y_train_bin, y_pred_train, y_pred_prob_train, name='Random_forest_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193


Аналогично, видим переобучение

# Bagging classifier (logistic regression)

In [ ]:
bg = BaggingClassifier(base_estimator=LogisticRegression(),
                      random_state=RAND)
bg.fit(pd.DataFrame(X_train_bin_scaled), y_train_bin)

BaggingClassifier(base_estimator=LogisticRegression(), random_state=10)

In [ ]:
y_pred = bg.predict(X_test_bin_scaled)
y_pred_prob = bg.predict_proba(X_test_bin_scaled)
metrics = metrics.append(get_metrics(y_test, y_pred, y_pred_prob, name='Bagging_classifier'))

y_pred_train = bg.predict(X_train_bin_scaled)
y_pred_prob_train = bg.predict_proba(X_train_bin_scaled)
metrics = metrics.append(
    get_metrics(y_train, y_pred_train, y_pred_prob_train, name='Bagging_classifier_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Bagging_classifier,0.973558,0.870698,0.744000,0.118622,0.204620,0.096452
0,Bagging_classifier_train,0.972204,0.879720,0.733333,0.112611,0.195240,0.098379


# XGBoost

In [ ]:
# Реализация f1-метрики для XGBoost
def f1_metric(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1 - f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [ ]:
# Выборки для проверки после каждой итерации обучения
X_train_, X_val, y_train_, y_val = train_test_split(X_train_bin,
                                                    y_train_bin,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)
eval_set = [(X_val, y_val)]

clf = XGBClassifier(random_state=RAND, scale_pos_weight = percent_of_negative_class)

clf.fit(X_train_,
        y_train_,
        eval_metric=f1_metric,
        eval_set=eval_set,
        early_stopping_rounds=100,
        verbose=2)

[0]	validation_0-error:0.027647	validation_0-f1_err:0.898161
Multiple eval metrics have been passed: 'validation_0-f1_err' will be used for early stopping.

Will train until validation_0-f1_err hasn't improved in 100 rounds.
[2]	validation_0-error:0.027778	validation_0-f1_err:0.908832
[4]	validation_0-error:0.027255	validation_0-f1_err:0.909884
[6]	validation_0-error:0.027255	validation_0-f1_err:0.909884
[8]	validation_0-error:0.028083	validation_0-f1_err:0.964126
[10]	validation_0-error:0.028083	validation_0-f1_err:0.964126
[12]	validation_0-error:0.028083	validation_0-f1_err:0.964126
[14]	validation_0-error:0.028083	validation_0-f1_err:0.964126
[16]	validation_0-error:0.028126	validation_0-f1_err:0.967066
[18]	validation_0-error:0.028126	validation_0-f1_err:0.967066
[20]	validation_0-error:0.028431	validation_0-f1_err:0.987897
[22]	validation_0-error:0.028387	validation_0-f1_err:0.984894
[24]	validation_0-error:0.028387	validation_0-f1_err:0.984894
[26]	validation_0-error:0.028387	va

XGBClassifier(random_state=10, scale_pos_weight=0.958)

In [ ]:
y_pred = clf.predict(X_test_bin)
y_pred_prob = clf.predict_proba(X_test_bin)
metrics = metrics.append(
    get_metrics(y_test_bin, y_pred, y_pred_prob, name='XGBoost'))

y_pred_train = clf.predict(X_train_)
y_pred_prob_train = clf.predict_proba(X_train_)
metrics = metrics.append(
    get_metrics(y_train_, y_pred_train, y_pred_prob_train, name='XGBoost_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Bagging_classifier,0.973558,0.870698,0.744000,0.118622,0.204620,0.096452
0,Bagging_classifier_train,0.972204,0.879720,0.733333,0.112611,0.195240,0.098379
0,XGBoost,0.973192,0.861562,0.904762,0.072704,0.134593,0.100149
0,XGBoost_train,0.971720,0.861385,0.879479,0.074115,0.136709,0.103957


# LightGBM

In [ ]:
# Реализация f1-метрики для LightGBM
def f1_metric(labels, scores):
    pred = np.round(scores)
    return 'f1', f1_score(labels, pred), True

In [ ]:
# Выборки для проверки после каждой итерации обучения
X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)
eval_set = [(X_val, y_val)]

clf = LGBMClassifier(random_state=RAND, scale_pos_weight=percent_of_negative_class)

eval_set = [(X_val, y_val)]

clf.fit(X_train_,
        y_train_,
        eval_metric=f1_metric,
        eval_set=eval_set,
        early_stopping_rounds=100,
        verbose=2)

Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.115064	valid_0's f1: 0.101449
[4]	valid_0's binary_logloss: 0.1111	valid_0's f1: 0.106936
[6]	valid_0's binary_logloss: 0.107991	valid_0's f1: 0.112392
[8]	valid_0's binary_logloss: 0.105188	valid_0's f1: 0.115108
[10]	valid_0's binary_logloss: 0.103158	valid_0's f1: 0.120516
[12]	valid_0's binary_logloss: 0.10142	valid_0's f1: 0.128388
[14]	valid_0's binary_logloss: 0.100046	valid_0's f1: 0.13881
[16]	valid_0's binary_logloss: 0.0989617	valid_0's f1: 0.148668
[18]	valid_0's binary_logloss: 0.0978376	valid_0's f1: 0.150628
[20]	valid_0's binary_logloss: 0.0964321	valid_0's f1: 0.164384
[22]	valid_0's binary_logloss: 0.0949505	valid_0's f1: 0.17415
[24]	valid_0's binary_logloss: 0.0942052	valid_0's f1: 0.188425
[26]	valid_0's binary_logloss: 0.0933336	valid_0's f1: 0.209549
[28]	valid_0's binary_logloss: 0.092577	valid_0's f1: 0.211921
[30]	valid_0's binary_logloss: 0.0919217	valid_0's f1: 0.2

LGBMClassifier(random_state=10, scale_pos_weight=0.958)

In [ ]:
y_pred = clf.predict(X_test)
y_pred_prob = clf.predict_proba(X_test)
metrics = metrics.append(
    get_metrics(y_test, y_pred, y_pred_prob, name='LightGBM'))

y_pred_train = clf.predict(X_train_)
y_pred_prob_train = clf.predict_proba(X_train_)
metrics = metrics.append(
    get_metrics(y_train_, y_pred_train, y_pred_prob_train, name='LightGBM_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Bagging_classifier,0.973558,0.870698,0.744000,0.118622,0.204620,0.096452
0,Bagging_classifier_train,0.972204,0.879720,0.733333,0.112611,0.195240,0.098379
0,XGBoost,0.973192,0.861562,0.904762,0.072704,0.134593,0.100149
0,XGBoost_train,0.971720,0.861385,0.879479,0.074115,0.136709,0.103957


# Catboost 

In [ ]:
cat_features = ['DISC_ID', 'TYPE_NAME', 'GENDER', 'CITIZENSHIP', 
            'EXAM_TYPE', 'EXAM_SUBJECT_1', 'EXAM_SUBJECT_2', 
            'EXAM_SUBJECT_3', 'ADMITTED_SUBJECT_PRIZE_LEVEL', 'REGION_ID']

clf = CatBoostClassifier(random_state=RAND,\
                         scale_pos_weight = percent_of_negative_class,
                         cat_features = cat_features)

eval_set = [(X_val, y_val)]

clf.fit(X_train_,
        y_train_,
        eval_set=eval_set,
        early_stopping_rounds=100,
        verbose=2)

Learning rate set to 0.10351
0:	learn: 0.5376462	test: 0.5370327	best: 0.5370327 (0)	total: 194ms	remaining: 3m 13s
2:	learn: 0.3442423	test: 0.3424119	best: 0.3424119 (2)	total: 618ms	remaining: 3m 25s
4:	learn: 0.2454176	test: 0.2427825	best: 0.2427825 (4)	total: 859ms	remaining: 2m 51s
6:	learn: 0.1890580	test: 0.1855906	best: 0.1855906 (6)	total: 1.19s	remaining: 2m 48s
8:	learn: 0.1599901	test: 0.1559084	best: 0.1559084 (8)	total: 1.44s	remaining: 2m 38s
10:	learn: 0.1427467	test: 0.1380386	best: 0.1380386 (10)	total: 1.81s	remaining: 2m 42s
12:	learn: 0.1297972	test: 0.1248159	best: 0.1248159 (12)	total: 2.14s	remaining: 2m 42s
14:	learn: 0.1235834	test: 0.1183105	best: 0.1183105 (14)	total: 2.49s	remaining: 2m 43s
16:	learn: 0.1181298	test: 0.1127023	best: 0.1127023 (16)	total: 2.85s	remaining: 2m 45s
18:	learn: 0.1155584	test: 0.1099959	best: 0.1099959 (18)	total: 3.18s	remaining: 2m 44s
20:	learn: 0.1133832	test: 0.1076488	best: 0.1076488 (20)	total: 3.52s	remaining: 2m 44s
22

In [ ]:
y_pred = clf.predict(X_test)
y_pred_prob = clf.predict_proba(X_test)
metrics = metrics.append(
    get_metrics(y_test, y_pred, y_pred_prob, name='Catboost'))

y_pred_train = clf.predict(X_train_)
y_pred_prob_train = clf.predict_proba(X_train_)
metrics = metrics.append(
    get_metrics(y_train_, y_pred_train, y_pred_prob_train, name='Catboost_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Bagging_classifier,0.973558,0.870698,0.744000,0.118622,0.204620,0.096452
0,Bagging_classifier_train,0.972204,0.879720,0.733333,0.112611,0.195240,0.098379
0,XGBoost,0.973192,0.861562,0.904762,0.072704,0.134593,0.100149
0,XGBoost_train,0.971720,0.861385,0.879479,0.074115,0.136709,0.103957


# Gradient boosting

In [ ]:
clf = GradientBoostingClassifier(random_state=RAND)
clf.fit(X_train_bin, y_train_bin)

GradientBoostingClassifier(random_state=10)

In [ ]:
y_pred = clf.predict(X_test_bin)
y_pred_prob = clf.predict_proba(X_test_bin)
metrics = metrics.append(
    get_metrics(y_test, y_pred, y_pred_prob, name='Gradient_Boosting'))

y_pred_train = clf.predict(X_train_bin)
y_pred_prob_train = clf.predict_proba(X_train_bin)
metrics = metrics.append(
    get_metrics(y_train_bin, y_pred_train, y_pred_prob_train, name='Gradient_Boosting_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Bagging_classifier,0.973558,0.870698,0.744000,0.118622,0.204620,0.096452
0,Bagging_classifier_train,0.972204,0.879720,0.733333,0.112611,0.195240,0.098379
0,XGBoost,0.973192,0.861562,0.904762,0.072704,0.134593,0.100149
0,XGBoost_train,0.971720,0.861385,0.879479,0.074115,0.136709,0.103957


# kNN

In [ ]:
clf = KNeighborsClassifier()
clf.fit(X_train_bin_scaled, y_train_bin)

KNeighborsClassifier()

In [ ]:
y_pred = clf.predict(X_test_bin_scaled)
y_pred_prob = clf.predict_proba(X_test_bin_scaled)
metrics = metrics.append(
    get_metrics(y_test_bin, y_pred, y_pred_prob, name='kNN'))

y_pred_train = clf.predict(X_train_bin_scaled)
y_pred_prob_train = clf.predict_proba(X_train_bin_scaled)
metrics = metrics.append(
    get_metrics(y_train_bin, y_pred_train, y_pred_prob_train, name='kNN_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,LogisticRegression,0.768753,0.876500,0.093498,0.812500,0.167698,0.438062
0,LogisticRegression_train,0.769455,0.889472,0.100677,0.844579,0.179908,0.437340
0,Decision_tree,0.969206,0.747928,0.466357,0.512755,0.488457,1.036513
0,Decision_tree_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Random_forest,0.974290,0.937556,0.651685,0.221939,0.331113,0.109669
0,Random_forest_train,0.998983,0.999997,0.967147,1.000000,0.983299,0.002193
0,Bagging_classifier,0.973558,0.870698,0.744000,0.118622,0.204620,0.096452
0,Bagging_classifier_train,0.972204,0.879720,0.733333,0.112611,0.195240,0.098379
0,XGBoost,0.973192,0.861562,0.904762,0.072704,0.134593,0.100149
0,XGBoost_train,0.971720,0.861385,0.879479,0.074115,0.136709,0.103957


# Выводы

Итак, качество базового алгоритма будем ориентироваться на:

1) метрику f1, т.к. она является критерием оценивания в соревновании

1) значение метрик roc_auc, logloss на тестовой выборке

2) разрыв между метриками f1, roc_auc, log_loss на тренировочной и тестовой выборках 

На другие метрики особо смотреть не будем, но их полезно выводить, чтобы лучше контролироовать адекватность происходящего. Так, recall, precision и занулятся, если все объекты выборки будут отнесены к одному классу. Это довольно редкая ситуация, поэтому обратить на неё внимание было бы неплохо. Единственной полностью бесполезной метрикой, конечно, является accuracy в силу дисбаланса классов

Исходя из этих соображений, по таблице можно понять, что лучшие результаты дали LightGBM и Catboost, для них мы и будем подбирать гиперпараметры